<a href="https://colab.research.google.com/github/mariel/Sisteme-Inteligente/blob/master/Nota_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

In [0]:
import matplotlib.pyplot as plt
plt.style.use("ggplot")

In [0]:
df = pd.read_csv("/content/pd_speech_features.csv")

In [0]:
df.head()

In [0]:
X = df.drop("class", axis = 1)
y = df['class']

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, shuffle = True)

In [0]:
sum(y_train),len(y_test)

In [0]:
from sklearn.preprocessing import MinMaxScaler

In [0]:
X_train

In [0]:
y_train

In [0]:
sc = MinMaxScaler((-1,1))

In [0]:
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [0]:
X_test

In [0]:
len(X_train)

In [0]:
from torch.utils.data import Dataset, DataLoader

In [0]:
import torch
from torch import nn,optim

In [0]:
trainDataset = Dataset(X_train,y_train)

In [0]:
trainDataset[0]

In [0]:
trainLoader = DataLoader(dataset= trainDataset,
                        batch_size = 32,
                         shuffle = True,
                         num_workers = 1)

In [0]:
validationDataset = Dataset(X_test, y_test)

In [0]:
validationLoader = DataLoader(dataset = validationDataset,
                              batch_size = 32,
                              shuffle = True,
                              num_workers=1)

In [0]:
class ParkinsonsNN(nn.Module):
    def __init__(self):
        super(ParkinsonsNN,self).__init__()

        self.sequential = nn.Sequential(
            nn.Linear(754,350),
            nn.ReLU(),
            nn.Linear(350, 200),
            nn.ReLU(),
            nn.Linear(200,175),
            nn.ReLU(),
            nn.Linear(175,100),
            nn.ReLU(),
            nn.Linear(100,60),
            nn.ReLU(),
            nn.Linear(60,2)
        )
    def forward(self,x):
      return self.sequential(x)

In [0]:
net = ParkinsonsNN()

In [0]:
optimizer = optim.SGD(net.parameters(), lr = 0.01)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,patience=5,verbose=True)

In [0]:
train_losses = []

In [0]:
accuracies = []

In [0]:
from sklearn.metrics import accuracy_score

In [0]:
def train(epoch):
  net.train()
  losses =[]
  
  for batch_idx, data in enumerate(trainLoader,0):
    inputs,labels = data
    outputs = net(inputs)
    loss = criterion(outputs,labels)

    losses.append(loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f"[Train Epoch: {epoch}, Batch: {batch_idx+1}, Loss:{loss.item()}]")
  mean_loss = sum(losses)/len(losses)
  scheduler.step(mean_loss)
  train_losses.append(mean_loss)
  print(f"[TRAIN] Epoch: {epoch} Loss:{mean_loss}")

In [0]:
test_losses=[]

In [0]:
def validation():
    net.eval()

    test_loss=[]
    correct = 0

    with torch.no_grad():
        for batch_idx, data in enumerate(validationLoader, 0):
          inputs, labels = data

          output=net(inputs)

          loss= criterion(output, labels)
          test_loss.append(loss.item())
 
          pred = output.data.max(1, keepdim=True)[1]

          correct += pred.eq(labels.data.view_as(pred)).sum()
          current_correct=pred.eq(labels.data.view_as(pred)).sum()          
          print("============")
          print(f"[Validation set] Batch index: {batch_idx+1} Batch loss: {loss.item()}, Accuracy: {100. * current_correct/len(inputs)}%")
          print("============")
        mean_loss=sum(test_loss)/len(test_loss)
        test_losses.append(mean_loss)
        accuracy = 100. * correct/len(validationLoader.dataset)
        print(f"[Validation set] Loss: {mean_loss}, Accuracy: {accuracy}%")
          
        accuracies.append(accuracy)

In [0]:
for epoch in range(150):
  train(epoch)
  validation()

In [0]:
plt.plot(train_losses, "r-", test_losses, "b-")

In [0]:
plt.plot(accuracies)

In [0]:
def try_a_single_example_with_the_network(index_from_the_validation_set):
  with torch.no_grad():
    pred_test = net(validationDataset[index_from_the_validation_set][0].view(1, -1))
    _, preds_y = torch.max(pred_test, 1)
    return preds_y

In [0]:
index_of_test=5

In [0]:
print(f"Neural network response is: {try_a_single_example_with_the_network(index_of_test).item()}")
print(f"Actual response is: {validationDataset[index_of_test][1].view(-1).item()}")